# 2025 세법 개정본 반영

In [ ]:
pip install jpype1

In [ ]:
%pip install tabula-py

In [ ]:
pip install --upgrade langchain chromadb

In [ ]:
pip install requests-cache

In [94]:
import os
import re
from tqdm import tqdm
from tabula import read_pdf
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.cache import SQLiteCache
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from dotenv import load_dotenv

load_dotenv()

pdf_file = "data/tax_etc/연말정산_신고안내.pdf"

# SQLite 캐시 설정
SQLiteCache(".cache_prompt_etc1.sqlite")

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 모든 PDF 처리
docs = []

# Tabula로 PDF에서 표 데이터 읽기
try:
    tables = read_pdf(pdf_file, pages="17-425", stream=True, multiple_tables=True) # pages="19-44,47-71,75-161"
    table_texts = [table.to_string(index=False, header=True) for table in tables]
except Exception as e:
    print(f"Tabula Error with {pdf_file}: {e}")
    table_texts = []

# PyMuPDFLoader로 텍스트 데이터 로드 및 페이지 범위 필터링링
try:
    text_loader = PyMuPDFLoader(pdf_file)
    texts = text_loader.load()

    for i, text in enumerate(texts):
        text.metadata["page"] = i + 1      

    page_ranges = [(17, 426)]
    texts = [
        text for text in texts
        if any(start <= text.metadata.get("page", 0) <= end for start, end in page_ranges)
    ]
except Exception as e:
    print(f"PyMuPDFLoader Error with {pdf_file}: {e}")
    texts = []

# 표 텍스트를 Document 형식으로 변환
table_docs = [
    Document(page_content=text, metadata={"source": f"{pdf_file} - Table {i + 1}"})
    for i, text in enumerate(table_texts)
]

    # 텍스트를 Document 형식으로 변환
text_docs = [
    Document(page_content=text.page_content, metadata={"source": pdf_file})
    for text in texts
]

    # 텍스트 문서와 표 문서를 결합
docs.extend(text_docs + table_docs)

# 정규식 패턴화
update_docs = []

pattern1 = (
    r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  
    r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"
    r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"
    r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"
    r"Ⅰ\. 근로소득\n\d+|"
    r"II\. 근로소득 원천징수 및 연말정산\n\d+|"
    r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"
    r"IV\. 특별소득공제(소법 §52)\n\d+|"
    r"V\. 그 밖의 소득공제(조특법)\n\d+|"
    r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"
    r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"
    r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"
    r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"
    r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"
    r"I\. 사업소득 연말정산\n\d+|"
    r"II\. 연금소득 연말정산\n\d+|"
    r"I\. 종교인소득이란?\n\d+|"
    r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"
    r"부록1\. 연말정산 관련 서비스\n\d+|"
    r"부록2\. 연말정산간소화 서비스\n\d+|"
    r"부록5\. 연말정산 주요 용어 설명\n\d+|"
    r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"
    r"\bNaN\b"
)
pattern2 =r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])"
pattern3 = r"\s+"

for doc in docs:
    if doc.page_content:
        edit_content = re.sub(pattern1, "", doc.page_content)
        edit_content = re.sub(pattern2, r"\1\2" , edit_content)
        edit_content = re.sub(pattern3, " ", edit_content)
    else:
        edit_content = ""
    
    updated_docs = Document(page_content=edit_content, metadata=doc.metadata)   
    update_docs.append(updated_docs)
    
# 텍스트를 청크로 분리
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o",
    chunk_size=2000,
    chunk_overlap=100,
)

split_docs = splitter.split_documents(docs)

# Vector Store 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="tax_etc",
    persist_directory="vector_store/chroma/tax_etc_2"
)

# Retriever 생성
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 10}
)

c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\AppData\Local\miniconda3\envs\langchain2\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(


In [87]:
docs[1]

Document(metadata={'source': 'data/tax_etc/연말정산_신고안내.pdf'}, page_content='원천징수의무자를 위한 \n2024년 연말정산 신고안내\n2\n3\n직무발명보상금 비과세 한도 상향 \n(소득세법 제12조, 같은 법 시행령 제17조의3, 제18조)\n<개정취지> 기술개발 유인 제고\n종          전\n개          정\n▢직무발명보상금에 대한 비과세\n▢ 비과세 한도 상향 및 적용범위 조정\n○(대상) 종업원, 교직원, 학생에게 지급하는 직무발명\n보상금*으로서 연 500만원 이하의 금액\n    * ｢발명진흥법｣ §2(2)에 따른 직무발명으로 받는보상금\n○연 500만원 → 연 700만원\n<신  설>\n   - 아래에 해당하는 종업원은 제외\n    ➊ 사용자가 개인사업자인 경우 : \n       해당 개인사업자 및 그와 친족관계에 있는 자\n    ➋ 사용자가 법인인 경우 : \n       해당 법인의 지배주주 등* 및 그와 특수관계**에 있는 자\n        * 법인세법 시행령 §43⑦에 따른 지배주주 등\n       ** 친족관계 또는 경영지배관계\n<적용시기> 2024.1.1. 이후 지급받는 소득분부터 적용\n')

In [98]:
update_docs[1]

Document(metadata={'source': 'data/tax_etc/연말정산_신고안내.pdf'}, page_content=' 3 직무발명보상금 비과세 한도 상향 (소득세법 제12조, 같은 법 시행령 제17조의3, 제18조) <개정취지> 기술개발 유인 제고종 전개 정 ▢직무발명보상금에 대한 비과세 ▢ 비과세 한도 상향 및 적용범위 조정 ○(대상) 종업원, 교직원, 학생에게 지급하는 직무발명보상금*으로서 연 500만원 이하의 금액 * ｢발명진흥법｣ §2(2)에 따른 직무발명으로 받는보상금 ○연 500만원 → 연 700만원 <신 설> - 아래에 해당하는 종업원은 제외 ➊ 사용자가 개인사업자인 경우 : 해당 개인사업자 및 그와 친족관계에 있는 자 ➋ 사용자가 법인인 경우 : 해당 법인의 지배주주 등* 및 그와 특수관계**에 있는 자 * 법인세법 시행령 §43⑦에 따른 지배주주 등 ** 친족관계 또는 경영지배관계 <적용시기> 2024.1.1. 이후 지급받는 소득분부터 적용 ')

In [99]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.
        
        {context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [104]:
# chain.invoke("장기주택저당차입금 이자상환액 소득공제 확대에 대해 알려줘")
# chain.invoke("원천징수이행상황신고서 및 지급명세서 제출 시 주의사항 알려줘")
chain.invoke("연말정산시 소득·세액공제 증명서류 중점 확인사항")

'연말정산 시 소득·세액공제 증명서류를 제출할 때, 원천징수의무자는 다음과 같은 항목을 중점적으로 확인해야 합니다. 이는 과다공제에 따른 가산세 부담을 방지하기 위함입니다.\n\n1. **인적공제**:\n   - 기본공제대상 부양가족을 새로 추가하는 경우, 중복공제 여부와 연간 소득금액 100만원(근로소득만 있는 자는 총급여 500만원) 초과 여부 확인 필요.\n   - 과세기간 개시일 전에 사망하거나 국외로 이주한 자는 기본공제 대상자가 아님.\n\n2. **주택자금공제**:\n   - 주택임차차입금 원리금상환액 공제 시, 주민등록표등본을 통해 세대주 여부 확인 및 임대차계약서와 금전소비대차계약서 확인 필요.\n   - 장기주택저당차입금 이자상환액 공제 시, 주택 소유 여부 및 대출 조건 등을 확인.\n\n3. **주택마련저축공제**:\n   - 세대주 여부 및 근로자 본인 명의 가입 여부 확인.\n\n4. **신용카드 소득공제**:\n   - 기본공제대상자인 형제자매의 신용카드 사용금액 제외 여부 확인.\n   - 연간소득금액이 100만원을 초과한 배우자 등의 신용카드 사용금액 제외 여부 확인.\n\n5. **연금계좌 세액공제**:\n   - 개인연금저축 납입액의 잘못된 신청 여부 및 중도해지 여부 확인.\n\n6. **보험료 세액공제**:\n   - 보장성보험료의 피보험자가 기본공제대상자인지 확인.\n\n7. **의료비 세액공제**:\n   - 기본공제대상자인 직계존비속 등의 의료비를 근로자가 직접 부담했는지 확인.\n   - 보전받은 의료비를 제외하였는지 여부 확인.\n\n8. **교육비 세액공제**:\n   - 자녀 학원비는 취학 전(입학연도 1월~2월까지)에 지출한 경우만 공제 가능.\n   - 근로자 본인을 위한 대학원 교육비인지 여부 확인.\n\n이러한 중점 확인사항은 과다공제를 방지하고, 정확한 세액공제를 받을 수 있도록 하기 위함입니다. 각 공제 항목에 대한 자세한 요건과 필요 서류를 정확히 확인하여 준비하는 것이 중요합니다.'